In [ ]:
import os
import site
os.sys.path.insert(0, '/home/schirrmr/braindecode/code/')
os.sys.path.insert(0, '/home/schirrmr/braindecode/code/braindecode/')
os.sys.path.insert(0, '/home/schirrmr/code/auto-diagnosis/')
%cd /home/schirrmr/
# switch to cpu
os.environ['THEANO_FLAGS'] = 'floatX=float32,device=cpu,nvcc.fastmath=True'

%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import cm
%matplotlib inline
%config InlineBackend.figure_format = 'png' 
matplotlib.rcParams['font.size'] = 7

import matplotlib.lines as mlines
import seaborn
seaborn.set_style('darkgrid')
matplotlib.rcParams['figure.figsize'] = (12.0, 3.0)
import logging
import importlib
importlib.reload(logging) # see https://stackoverflow.com/a/21475297/1469195
log = logging.getLogger()
log.setLevel('DEBUG')
import sys
logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                     level=logging.DEBUG, stream=sys.stdout)
from numpy.random import RandomState
from braindecode.torch_ext.util import np_to_var
from autodiag.perturbation import combine_covs, combine_vars

In [ ]:
import re

In [ ]:
from nltk import word_tokenize, regexp_tokenize

In [ ]:
import nltk

In [ ]:
from autodiag.dataset import get_all_sorted_file_names_and_labels
from autodiag.count_words import compute_counts_and_relative_frequencies, compute_imprs_word_counts

In [ ]:
file_names, labels = get_all_sorted_file_names_and_labels('eval')
file_names = np.array(file_names)

In [ ]:

result = compute_counts_and_relative_frequencies(file_names, np.zeros(len(labels)),
                                                 correct_mask=labels == 0,
                                       label=0)

In [ ]:
n_words = sum(list(result.incorrect_counter.values()))

In [ ]:
all_words_freqs = [(word, result.incorrect_counter[word] / float(n_words)) for word in result.incorrect_counter]

In [ ]:
all_words_freqs = sorted(all_words_freqs, key=lambda t: t[1])

In [ ]:
for word, freq in all_words_freqs[::-1][:10]:
    print(word, freq)

In [ ]:


print("\n ----------------------------------------------\n")
for key, rel_freq in result.freq_ratios[::-1][:20]:
    if rel_freq < 3:
        break
    print("{:20s} {:5.2f} {:d} {:d}".format(key, rel_freq, 
                                            result.correct_counter[key] > 0,
                                            result.incorrect_counter[key] > 0))

### incorrect vs correct

In [ ]:
trial_preds = np.load('data/models/pytorch/auto-diag/6000-input/21/test_trial_preds.npy')
trial_labels = np.load('data/models/pytorch/auto-diag/6000-input/21/test_trial_labels.npy')

In [ ]:
correct_mask = np.argmax(trial_preds, axis=1) == trial_labels

In [ ]:
print(str(sum(~correct_mask & (labels == 1))) + " errors\n")

result = compute_counts_and_relative_frequencies(file_names, labels, correct_mask=correct_mask,
                                       label=1)

for key, rel_freq in result.freq_ratios:
    if rel_freq > 0.5:
        break
    print("{:20s} {:5.2f} {:d} {:d}".format(key, rel_freq, result.correct_counter[key] > 0, 
                                            result.incorrect_counter[key] > 0))

print("\n ----------------------------------------------\n")
for key, rel_freq in result.freq_ratios[::-1][:20]:
    if rel_freq < 3:
        break
    print("{:20s} {:5.2f} {:d} {:d}".format(key, rel_freq, 
                                            result.correct_counter[key] > 0,
                                            result.incorrect_counter[key] > 0))
print("\n ----------------------------------------------\n\n\n")
    
for key, freq_diff in result.freq_diffs:
    if freq_diff > -0.005:
        break
    print("{:20s} {:5.2f} {:d} {:d}".format(key, freq_diff*100, result.correct_counter[key] > 0, 
                                            result.incorrect_counter[key] > 0))

print("\n ----------------------------------------------\n")
for key, freq_diff in result.freq_diffs[::-1][:20]:
    if freq_diff < 0.005:
        break
    print("{:20s} {:5.2f} {:d} {:d}".format(key, freq_diff*100, 
                                            result.correct_counter[key] > 0,
                                            result.incorrect_counter[key] > 0))

In [ ]:
[print(impr +"\n") for impr in result.correct_imprs];

In [ ]:
[print(impr +"\n") for impr in result.incorrect_imprs];

In [ ]:
print(str(sum(~correct_mask & (labels == 0))) + " errors\n")

result = compute_counts_and_relative_frequencies(file_names, labels,correct_mask=correct_mask,
                                       label=0)

for key, rel_freq in result.freq_ratios:
    if rel_freq > 0.4:
        break
    print("{:20s} {:5.2f} {:d} {:d}".format(key, rel_freq, result.correct_counter[key] > 0, 
                                            result.incorrect_counter[key] > 0))

print("\n ----------------------------------------------\n")
for key, rel_freq in result.freq_ratios[::-1][:20]:
    if rel_freq < 3:
        break
    print("{:20s} {:5.2f} {:d} {:d}".format(key, rel_freq, 
                                            result.correct_counter[key] > 0,
                                            result.incorrect_counter[key] > 0))
print("\n ----------------------------------------------\n\n\n")
    
for key, freq_diff in result.freq_diffs:
    if freq_diff > -0.005:
        break
    print("{:20s} {:5.2f} {:d} {:d}".format(key, freq_diff*100, result.correct_counter[key] > 0, 
                                            result.incorrect_counter[key] > 0))

print("\n ----------------------------------------------\n")
for key, freq_diff in result.freq_diffs[::-1][:20]:
    if freq_diff < 0.005:
        break
    print("{:20s} {:5.2f} {:d} {:d}".format(key, freq_diff*100, 
                                            result.correct_counter[key] > 0,
                                            result.incorrect_counter[key] > 0))

In [ ]:
[print(impr +"\n") for impr in result.correct_imprs];

In [ ]:
result.correct_counter['as']

In [ ]:
[print(impr +"\n") for impr in result.incorrect_imprs];

In [ ]:
[print(impr +"\n") for impr in correct_abnormal_imprs];

In [ ]:
correct_normal = correct_mask & (labels == 0)
correct_normal_texts = extract_texts(file_names[np.flatnonzero(correct_normal)])
correct_normal_dicts = [extract_dict(t) for t in correct_normal_texts]

incorrect_abnormal = (~correct_abnormal) & (labels == 1)
incorrect_abnormal_texts = extract_texts(file_names[np.flatnonzero(incorrect_abnormal)])
incorrect_abnormal_dicts = [extract_dict(t) for t in incorrect_abnormal_texts]

incorrect_abnormal_imprs = [d['IMPRESSION'] for d in incorrect_abnormal_dicts]

correct_abnormal_imprs = [d['IMPRESSION'] for d in correct_abnormal_dicts]
compare_freqs = []
incorrect_counter = Counter(incorrect_abnormal_words)
correct_counter = Counter(correct_abnormal_words)


for key in set(incorrect_counter.keys()) | set(correct_counter.keys()):
    if (incorrect_counter[key] > 1) or (correct_counter[key] > 1):
        rel_freq = np.maximum(incorrect_counter[key],1) / np.maximum(correct_counter[key], 1)
        rel_freq *= len(correct_abnormal_words) / float(len(incorrect_abnormal_words))
        compare_freqs.append((key, rel_freq))

sorted_freqs = sorted(compare_freqs, key=lambda t: t[1])